In [1]:
!pip install --upgrade pip
!pip install --upgrade --force-reinstall tflite_model_maker
!sudo apt-get -y install libusb-1.0-0-dev
!pip install pycocotools
!pip install gdown

     |████████████████████████████████| 2.1 MB 15.0 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.2.4
    Uninstalling pip-20.2.4:
      Successfully uninstalled pip-20.2.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.3/577.3 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 54.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 69.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 60.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 63.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 45.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━

In [1]:
import os
from datetime import datetime

from math import floor
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow as tf
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
from numba import cuda
from absl import logging

import yaml
import PIL

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [6]:
!apt-get zip unzip
!gdown 1tq7vkIzMnezhOsWurwRYEaaQ-QhvQpFK
!unzip vedai_corrected.zip

Downloading...
From: https://drive.google.com/uc?id=1tq7vkIzMnezhOsWurwRYEaaQ-QhvQpFK
To: /root/vedai_corrected.zip
100%|██████████████████████████████████████| 1.28G/1.28G [00:22<00:00, 58.0MB/s]


In [8]:
!nvidia-smi

Mon Dec 12 13:26:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:47:00.0 Off |                  N/A |
| 95%    8C    P8    48W / 350W |      1MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
device = cuda.get_current_device() 
device.reset()

In [3]:
def decode_img(img_path, img_height=1024, img_width=1024, convert_dtype=True):
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [img_height, img_width])
    if convert_dtype:
        img = tf.image.convert_image_dtype(img, dtype=tf.uint8, saturate=False)
    img = tf.reshape(img, [1, img_height, img_width, 3])
    # Resize the image to the desired size
    return img

def create_nms_bbox(test_img, detector_output):
    suppressed_idx = tf.image.non_max_suppression(
    tf.squeeze(detector_output["detection_boxes"]),
    tf.squeeze(detector_output["detection_scores"]),
    max_output_size=tf.constant(500),
    iou_threshold=0.3,
    score_threshold=float('-inf'),
    name=None
    )

    bounding_boxes_suppressed = tf.gather(tf.squeeze(detector_output["detection_boxes"]), suppressed_idx).numpy().squeeze()
    labels_suppressed = tf.gather(tf.squeeze(detector_output["detection_classes"]), suppressed_idx).numpy().astype(int).astype(str).squeeze().tolist()
    draw_bounding_boxes_on_image(test_img, bounding_boxes_suppressed, color='red', thickness=4, display_str_list_list=labels_suppressed)
    return test_img

# VEDAI Dataset creation

In [4]:
CORRECTED_PATH = os.path.join("vedai_corrected")
ANNOTATIONS_DIR = os.path.join(CORRECTED_PATH, "annotations")
ANNOTATIONS_MERGED_FILE_PATH = os.path.join(CORRECTED_PATH, "merged_annotations.csv")
IMAGES_DIR = os.path.join(CORRECTED_PATH, "images")
BACKBONE = 'efficientdet_lite4'
CLASSES_DICT = {1: "car",
                2: "truck",
                3: "pickup",
                4: "tractor",
                5: "camping car",
                6: "boat",
                7: "motorcycle",
                8: "bus",
                9: "van",
                10: "other",
                11: "small plane",
                12: "large plane"}

def merge_annotation_folder(annotations_dir, classes_dict, train_proportion=0.8, test_proportion=0.1):
    if train_proportion+test_proportion>=1:
        raise ValueError("Input a train and test proportion summing up to strictly less than 1.")
    annotation_files = list(Path(annotations_dir).rglob('*.txt'))
    indices = list(range(len(annotation_files)))
    files = [str(idx) + '.txt' for idx in indices]
    # else:
    #   files = [filename for filename in list(sorted(os.listdir(self.annotations_dir))) if filename.endswith('.txt')]
    #   indices = [convert_id_to_idx(filename.replace('.txt', '')) for filename in files]
    abs_filepaths = [os.path.join(annotations_dir, annotation_file) for annotation_file in files]
    annotations = pd.DataFrame(columns=["dataset", "x", "y", "width", "length", "idx", "empty"])
    # for img_file, filepath in zip(files, abs_filepaths):
    for idx, img_file in zip(indices, abs_filepaths):
        temp_annotation = pd.read_csv(img_file, sep=' ', names=["x", "y", "width", "length"]).reset_index(drop=False)
    # temp_annotation["image_id"] = img_file.split('.')[0]
    temp_annotation["idx"] = idx
    annotations = pd.concat([annotations, temp_annotation])
    annotations = annotations.rename(columns={"index":"labels"})
    annotations["labels"] = (annotations["labels"] + 1).astype(int)
    # annotations.index.name = None
    annotations["labels_name"] = annotations["labels"].replace(classes_dict)
    annotations["x_min"] = (annotations["x"] - annotations["width"]/2)
    annotations["y_min"] = (annotations["y"] - annotations["length"]/2)
    annotations["x_max"] = (annotations["x"] + annotations["width"]/2)
    annotations["y_max"] = (annotations["y"] + annotations["length"]/2)
    annotations["image_path"] = annotations["idx"].astype(str) + ".jpg"
    # Train test split
    images_idx = annotations.index.unique()
    train_idx = np.random.choice(images_idx, replace=False, size=floor(train_proportion*len(images_idx)))
    test_idx = np.random.choice(list(set(images_idx) - set(train_idx)), replace=False, size=floor(test_proportion*len(images_idx)))
    validation_idx = list(set(images_idx) - set(train_idx) - set(test_idx))
    annotations.loc[train_idx, "dataset"] = "TRAIN"
    annotations.loc[test_idx, "dataset"] = "TEST"
    annotations.loc[validation_idx, "dataset"] = "VALIDATION"
    annotations = annotations[["dataset", "image_path", "labels_name", "x_min", "y_min", "empty", "empty", "x_max", "y_max", "empty", "empty"]]
    return annotations

if not os.path.exists(ANNOTATIONS_MERGED_FILE_PATH):
    annotations = merge_annotation_folder(ANNOTATIONS_DIR, CLASSES_DICT, train_proportion=0.8, test_proportion=0.1)
    annotations.to_csv(ANNOTATIONS_MERGED_FILE_PATH, index=False, header=False)

In [ ]:
# def create_tf_example(example):
#   # TODO(user): Populate the following variables from your example.
#   height = None # Image height
#   width = None # Image width
#   filename = None # Filename of the image. Empty if image is not from file
#   encoded_image_data = None # Encoded image bytes
#   image_format = None # b'jpeg' or b'png'

#   xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
#   xmaxs = [] # List of normalized right x coordinates in bounding box
#              # (1 per box)
#   ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
#   ymaxs = [] # List of normalized bottom y coordinates in bounding box
#              # (1 per box)
#   classes_text = [] # List of string class name of bounding box (1 per box)
#   classes = [] # List of integer class id of bounding box (1 per box)

#   tf_example = tf.train.Example(features=tf.train.Features(feature={
#       'image/height': dataset_util.int64_feature(height),
#       'image/width': dataset_util.int64_feature(width),
#       'image/filename': dataset_util.bytes_feature(filename),
#       'image/source_id': dataset_util.bytes_feature(filename),
#       'image/encoded': dataset_util.bytes_feature(encoded_image_data),
#       'image/format': dataset_util.bytes_feature(image_format),
#       'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
#       'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
#       'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
#       'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
#       'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
#       'image/object/class/label': dataset_util.int64_list_feature(classes),
#   }))
#   return tf_example

# TF Lite - Detection Module

In [5]:
BATCH_SIZE = 32
IMG_HEIGHT = 1024
IMG_WIDTH = 1024
EPOCHS = 250
EXPORT_NAME = f"tflite_detector_{BACKBONE}_{str(datetime.now().date())}"

In [6]:
tf.get_logger().setLevel('ERROR')
logging.set_verbosity(logging.ERROR)

In [7]:
# !rm -rf /tmp/tmp*

In [8]:
# TO-DO : generate TFRecord files to accelerate loading
train_data, validation_data, test_data = object_detector.DataLoader.from_csv(os.path.join(ANNOTATIONS_MERGED_FILE_PATH), images_dir=IMAGES_DIR)

In [9]:
spec = model_spec.get(BACKBONE)
# spec.config.image_size = "1024x1024"
spec.config.num_classes = 13
spec.config.label_map = CLASSES_DICT

2022-12-12 15:49:28.841290: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 15:49:29.385969: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22311 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:47:00.0, compute capability: 8.6


In [10]:
model = object_detector.create(train_data, model_spec=spec, batch_size=BATCH_SIZE, train_whole_model=False, epochs=EPOCHS, validation_data=validation_data)

Epoch 1/250


2022-12-12 15:50:29.829129: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


38/38 [==============================] - 104s 1s/step - det_loss: 2.2273 - cls_loss: 1.4953 - box_loss: 0.0146 - reg_l2_loss: 0.0086 - loss: 2.2359 - learning_rate: 0.0240 - gradient_norm: 0.9116
Epoch 2/250
38/38 [==============================] - 49s 1s/step - det_loss: 1.4243 - cls_loss: 0.9712 - box_loss: 0.0091 - reg_l2_loss: 0.0088 - loss: 1.4331 - learning_rate: 0.0400 - gradient_norm: 1.5583
Epoch 3/250
38/38 [==============================] - 60s 2s/step - det_loss: 1.2680 - cls_loss: 0.8610 - box_loss: 0.0081 - reg_l2_loss: 0.0091 - loss: 1.2771 - learning_rate: 0.0400 - gradient_norm: 1.5408
Epoch 4/250
38/38 [==============================] - 49s 1s/step - det_loss: 1.2258 - cls_loss: 0.8527 - box_loss: 0.0075 - reg_l2_loss: 0.0092 - loss: 1.2351 - learning_rate: 0.0400 - gradient_norm: 1.4521
Epoch 5/250
38/38 [==============================] - 60s 2s/step - det_loss: 1.1812 - cls_loss: 0.8044 - box_loss: 0.0075 - reg_l2_loss: 0.0094 - loss: 1.1906 - learning_rate: 0.0400 

In [11]:
config = QuantizationConfig.for_float16()
model.export(export_dir=f'./{EXPORT_NAME}', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.TFLITE], quantization_config=config)

!apt-get install zip
!zip -r {EXPORT_NAME}.zip {EXPORT_NAME}

2022-12-12 20:20:39.059571: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-12-12 20:21:28.134302: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2022-12-12 20:21:41.317521: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-12-12 20:21:41.317578: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-12-12 20:21:41.322373: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpluxeyusx
2022-12-12 20:21:41.536720: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-12-12 20:21:41.541441: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if pre

/bin/bash: zip: command not found


NameError: name 'files' is not defined

In [24]:
!zip -r tflite_detector.zip tflite_detector_efficientdet_lite4

  adding: tflite_detector_efficientdet_lite4/ (stored 0%)
  adding: tflite_detector_efficientdet_lite4/model.tflite (deflated 10%)
  adding: tflite_detector_efficientdet_lite4/saved_model/ (stored 0%)
  adding: tflite_detector_efficientdet_lite4/saved_model/variables/ (stored 0%)
  adding: tflite_detector_efficientdet_lite4/saved_model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: tflite_detector_efficientdet_lite4/saved_model/variables/variables.index (deflated 84%)
  adding: tflite_detector_efficientdet_lite4/saved_model/assets/ (stored 0%)
  adding: tflite_detector_efficientdet_lite4/saved_model/saved_model.pb (deflated 89%)


In [ ]:
model.evaluate(test_data)

# TF Classical

In [ ]:
tf_image = decode_img(TEST_IMG_PATH)
image = ...  # A batch of preprocessed images with shape [batch_size, height, width, 3].
base_model = hub.KerasLayer("https://tfhub.dev/tensorflow/efficientdet/lite4/feature-vector/1")
cls_outputs, box_outputs = base_model(image, training=training)

In [ ]:

detector_output = efficient_det_2(tf_image)

test_img = PIL.Image.open(TEST_IMG_PATH)

test_img = create_nms_bbox(test_img, detector_output)
test_img

In [ ]:
m = tf.keras.Sequential([hub.KerasLayer("efficientdet_d6_1", trainable=True),
    tf.keras.layers.Dense(13, activation='softmax')
])

In [ ]:
m.build([1, 1024, 1024, 3])
m.summary()

In [ ]:
efficient_det_2 = hub.load("https://tfhub.dev/tensorflow/efficientdet/d6/1")

In [ ]:
tf.keras.utils.plot_model(
    efficient_det_2,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
)

In [ ]:
# model_test = tf.load("detection_retinanet_spinenet-96.tar")
with open("spinenet96_retinanet.yaml", "r") as file:
    model_config = yaml.load(file, Loader=yaml.FullLoader)